# Búsqueda con expresiones regulares

Las expresiones regulares o regex son de gran utilidad para hacer búsqueda de patrones dentro de textos. Estas están asociadas a los lenguajes regulares y, por tanto, a los autómatas finitos.
En este notebook, revisamos la implementación de un autómata que procesa una regex a partir de una adaptación del algoritmo de construcción de Thompson. 

### Construcción del autómata

En primer lugar, determinamos un objeto que definirá al autómata. Este se define como $A = (Q, q_0, F, \delta)$, donde $Q$ es un conjunto de estados, $q_0 \in Q$ el estado inicial, $F$ un conjunto de estados finales y $\delta$ la función de transición. 

Estos elementos definirán a la clase autómata que, además, cuenta con la <b>función de aceptación</b> de una cadena que regresa un valor boolean dependiendo si la cadena es aceptada o no por el autómata.
También definimos funciones de <b>match</b> que señalan, dentro de un documento, si se encuentra o no el patrón buscado.

In [1]:
class Automata(object):
    def __init__(self, states, initial, final, transitions):
        """
        Clase para objeto Autómata finito.
        
        Arguments
        ---------
        
        states : list
            Lista de estados que conforman el autómata
        initial : int
            Estado inicial
        final : list
            Lista de estados finales
        transitions : dict
            Dictionario de transicione {q: {sym: q'}}
        """
        self.states = states
        self.initial = initial
        self.final = final
        self.transitions = transitions
        
    def __str__(self):
        return '->'.join([str(q)+'-'+str(t) for q, t in self.transitions.items()])
    
    def accepts(self,string):
        """
        Aplicación de autómata a cadena.
        
        Arguments
        ---------
        
        string : str
            Cadena de entrada para procesar por el autómata
        
        Returns
        ------
        boolean
            True si es aceptada la cadena, False en otro caso.
        """
        #Inicializa el estado
        q = self.initial
        for sym in string:
            #Si no se cumple la transición, 
            #no se acepta la cadena
            try:
                #Transita a siguiente estado
                #cuando esta en la función
                q = self.transitions[q][sym]
            except:
                return False
        
        #Revisa si ha llegado a un estado final
        return q in self.final
    
    def match_token(self, tokens):
        """
        Encuentra el token que corresponde al patrón dentro de una documento tokenizado.
        
        Argumentos
        ----------
        tokens : list[str]
            Lista de tokens en donde se quiere buscar el patrón.
        
        Returns
        -------
            Lista de índices que indican los tokens que corresponden al patrón.
        """
        #Inicializa índice
        i = 0
        #Guarda localizaciones
        locs = []
        for token in tokens:
            #Aplica la función de aceptación
            output = self.accepts(token)
            #Si el autómata acepta la cdena
            if output == True:
                #Guarda el índice
                locs.append(i)
            #Avanza uno en el índice
            i += 1
        
        return locs
    
    def match(self, text):
        """
        Busca el patrón del autómata dentro de un texto.
        
        Arguments
        ---------
        text : str
            Texto (cadena) donde se buscará el patrón
        
        Returns
        -------
        locs : list
            Lista de las posiciones de inicio y final del patrón.
        """
        #Posición inicial en el texto
        pos = 0
        #Estado inicial del autómata
        q = self.initial
        #Guarda las posiciones
        locs = []
        for sym in text:
            #Guarda la posición cuando encuentra el primer símbolo
            first_sym = list(self.transitions[self.initial].keys())
            if sym in first_sym:
                #Guarda la posición cuando
                #ha encontrada un inicio del patrón
                begin = pos
            #Intenta aplicar el autómata
            try:
                #Siguiente transición
                q = self.transitions[q][sym]
                #Si llega a un estado final
                #guarda la posición donde acaba el patrón
                if q in self.final:
                    locs.append((begin, pos+1))
            except:
                #Si no encuentra el patrón
                #continua leyendo la cadena
                pass
            #Avanza a la siguiente posición
            #en el texto
            pos += 1
            
        return locs

Podemos definir un autómata muy simple y ver cuáles son las cadenas que es capaz de aceptar.

In [2]:
#Función de transiciones 
#Se usa formato de diccionario
delta = {0:{'j':1, 'h':1}, 1:{'a':0, 'e':0}}

#Definimos el autómata
fa = Automata(list(delta.keys()), 0, [0], delta)
print(fa)

#Procemos cadenas para ver si son aceptadas
print(fa.accepts('jajaja'))
print(fa.accepts('jaa'))

0-{'j': 1, 'h': 1}->1-{'a': 0, 'e': 0}
True
False


La búsqueda dentro del texto puede hacerse de forma lineal, revisando todo el documento como una cadena completa. En este caso, se recorrerá caracter por caracter hasta encontrar una sucesión de caracteres que cumpla por el patrón especificado por el autómata.

In [3]:
#Buscamos el patrón descrito por
#el autómata dentro de un texto
text = 'es jamón jajehe'
matches = fa.match(text)

print(matches)

#Podemos localizar las coincidencias
#dentro del texto
for begin,end in matches:
    print(text[begin:end])

[(3, 5), (9, 11), (11, 13), (13, 15)]
ja
ja
je
he


Como señalamos, los documentos se componen de tokens, $d = \{w_1, w_2, ...,w_n\}$, por lo que otra forma de buscar dentro de los documentos será encontrando un token que coincida con el patrón definido por el autómata, o en otras palabras, encontrando un token que sea aceptado por el autómata.

Para realizar esta búsqueda, definiremos una función simple de tokenización:

In [4]:
def clean(string):
    """
    Función simple que revisa si los caracteres son alfanuméricos.
    
    Arguments
    ---------
    string : str
        Cadena de entrada que se limpiará
    
    Returns
    -------
    Cadena sin caracterés alfanuméricos.
    """
    
    return ''.join(c for c in string if c.isalnum())

def tokenize(text):
    """
    Función simple de tokenización por espacios en blanco.
    
    Arguments
    ---------
    text : documentos
        Lista de documentos que se van a tokenizar.
        
    Returns
    -------
    tokens 
        La lista de los documentos tokenizada.
    """
    #Elimina \n de más
    #Pasa todo a minúscula
    lower_text = text.strip().lower()
    #Separa por espacio en blanco y aplica clean()
    tokens = [clean(word) for word in lower_text.split()]
    
    return tokens

Tokenizando el texto, podemos entonces aplicar la función para buscar aquellos tokens que sean aceptados por el autómata. Obtendremos la posición de estos tokens dentro del documento.

In [5]:
#Texto tokenizado
tokenized_text = tokenize(text)
print(tokenized_text)

#Encuentra los tokens que cumplen
fa.match_token(tokenized_text)

['es', 'jamón', 'jajehe']


[2]

## Algoritmo de construcción de Thompson

El algoritmo de construcción de Thompson construye un autómata a partir de una cadena que representa una expresión regular. Podemos decir que este algoritmo compila la expresión regular y regresa el autómata que le corresponde (generalmente no determinístico).

En este caso, hemos implenetado este algoritmo para símbolos de regex comúnes, como ?, *, +, | y concatenación. Sin embargo, puede extenderse a otros símbolos comunes en las regex. Asimismo, no implementamos el uso uso de corchetes en esta función, lo que limita las expresiones que podemos compilar.

In [6]:
def compile(regex):
    """
    Algoritmo de Construcción de Thompson para compilar expresiones regulares.
    
    Arguments
    ---------
    regex : str
        Expresión regular a compilar
        
    Returns
    -------
    Autómata finito (no determinístico) de la expresión regular.
    """
    #Posición en la regex
    pos = 0
    #Guarda la función de transición
    delta = {}
    #Pila para autómatas
    aut = []    
    for sym in regex:
        #Regex ?
        if sym == '?':
            #Observa el autómata previo
            prev_nfa = aut.pop()
            #Guarda la función de transición
            delta = prev_nfa.transitions
            #Guarda el último elemento
            last = len(delta)-1
            
            #Agrega una transición epsilon a la transición previa
            delta[last] = {**{'':last+1},**delta[last]}
            #Crea nuevo autómata y lo gaurda en pila
            nfa = Automata(list(delta.keys()), nfa.initial, [last,last+1], delta)
            aut.append(nfa)
        
        #Regex *
        #Símbolo previo cero o más veces
        elif sym == '*':
            #Observa el autómata previo
            prev_nfa = aut.pop()
            #Guarda la función de transición
            delta = prev_nfa.transitions
            #Guarda el último elemento
            last = len(delta)-1
            
            #Agrega función (q,e,q') (q',e,q'') y (q',s,q')
            delta[last] = {'':last+1}
            delta[last+1] = {'':last+2, regex[pos-1]:last+1}
            #Crea nuevo autómata y lo gaurda en pila
            nfa = Automata(list(delta.keys()), nfa.initial, [last+2], delta)
            aut.append(nfa)
            
        #Regex +
        #Símbolo anterior una o más veces
        elif sym == '+':
            #Observa el autómata previo
            prev_nfa = aut.pop()
            #Guarda la función de transición
            delta = prev_nfa.transitions
            #Guarda el último elemento
            last = len(delta)-1
            
            #Agrega reglas (q,e,q') y (q,s,q)
            delta[last] = {'':last+1, regex[pos-1]:last}
            #Crea nuevo autómata y lo gaurda en pila
            nfa = Automata(list(delta.keys()), nfa.initial, [last+1], delta)
            aut.append(nfa)

        #Regex s1|s2
        #Transita por s1 or s2
        elif sym == '|':
            #Observa el autómata previo
            prev_nfa = aut.pop()
            #Guarda la función de transición
            delta = prev_nfa.transitions
            #Guarda el estado más alto
            last = max(delta.keys())
            
            #La transiciónes son (q,s1,q') y (q,s2,q')
            delta[last] = {**delta[last],**{regex[pos+1]:last+1}}
            #Crea nuevo autómata y lo gaurda en pila
            nfa = Automata(list(delta.keys()), nfa.initial, [last+1], delta)
            aut.append(nfa)
            
        #Cualquier símbolo del alfabeto
        elif regex[pos-1] != '|':
            #Concatenación
            #Si hay símbolos previos
            if aut != []:
                #Observa el autómata previo
                prev_nfa = aut.pop()
                #Guarda la función de transición
                delta = prev_nfa.transitions
                #Guarda el estado más alto
                last = max(delta.keys())+1
                
                #Agrega regla de transición (q',s,q'')
                delta[last] = {sym:last+1}
                #Crea nuevo autómata y lo gaurda en pila
                nfa = Automata(list(delta.keys()), nfa.initial, [last+1], delta)
                aut.append(nfa)

            #Si es el único o primer símbolo
            else:
                #Estado inicial
                initial = len(delta)
                #Agrega regla (q,a,q')
                delta[initial] = {sym:initial+1}
                #Crea autómata y lo gaurda en pila
                nfa = Automata(list(delta.keys()), 0, [len(delta)], delta)
                aut.append(nfa)            
        
        #Avanza en la posición
        #de la regex
        pos += 1
    
    return aut[-1]

Podemos entonces aplicar el algoritmo a una expresión regular y observar que nos regresa un autómata que representa a esta expresión.

In [7]:
#Construye el autómata de la regex
pattern = compile('niña|os?')

print('Autómata {}\nEstados finales: {}'.format(pattern, pattern.final))

Autómata 0-{'n': 1}->1-{'i': 2}->2-{'ñ': 3}->3-{'a': 4, 'o': 4}->4-{'': 5, 's': 5}
Estados finales: [4, 5]


También podemos ver que tipo de cadenas acepta y cuales rechaza.

In [8]:
#Cadenas que acepta
print(pattern.accepts('niña'))
print(pattern.accepts('niño'))
print(pattern.accepts('niñas'))
print(pattern.accepts('niños'))
print(pattern.accepts('niñs'))

True
True
True
True
False


Finalmente, podemos aplicar una búsqueda en un documento tokenizado para ver en dónde se encuentran los tokens que corresponden al patrón que el autómata describe.

In [9]:
#Tokenización del texto
text = tokenize('Las niñas jugaban con el niño en el patio.')

#Encuentra las correspondecias
matches = pattern.match_token(text)

#Imprime índices y tokenes
#que cumplen el patrón dentro del texto
for i in matches:
    print(i, text[i])

1 niñas
5 niño
